In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

path_root = '../..'
sys.path.append(f'{path_root}/')
from src.util.functions import Functions as F

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = f'{path_root}/data'
path_csv = f'{path_data}/csv'

csv_out_name = f'{path_csv}/covid19-{level}.csv'
pop_regions_name = f'{path_csv}/popolazione-regioni.csv'
icu_regions_name = f'{path_csv}/terapie-intensive.csv'
regions_name = f'{path_csv}/regioni.csv'

In [2]:
url_cum = F.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,note,ingressi_terapia_intensiva,note_test,note_casi,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9004,2021-04-27T17:00:00,ITA,19,Sicilia,38.115697,13.362357,1254,168,1422,24663,...,NaN,4.0,NaN,NaN,205585.0,0.0,2253941.0,1555226.0,ITG,ITG1
9005,2021-04-27T17:00:00,ITA,9,Toscana,43.769231,11.255889,1449,257,1706,20060,...,NaN,21.0,NaN,NaN,221246.0,2827.0,3280399.0,758594.0,ITI,ITI1
9006,2021-04-27T17:00:00,ITA,10,Umbria,43.106758,12.388247,164,33,197,2801,...,"Si fa presente che 6 dei ricoveri NON UTI, non...",4.0,NaN,NaN,54036.0,0.0,851029.0,291308.0,ITI,ITI2
9007,2021-04-27T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,48,8,56,727,...,NaN,0.0,NaN,NaN,10285.0,501.0,89201.0,22706.0,ITC,ITC2


In [4]:
list(df_raw_r.columns)

['data',
 'stato',
 'codice_regione',
 'denominazione_regione',
 'lat',
 'long',
 'ricoverati_con_sintomi',
 'terapia_intensiva',
 'totale_ospedalizzati',
 'isolamento_domiciliare',
 'totale_positivi',
 'variazione_totale_positivi',
 'nuovi_positivi',
 'dimessi_guariti',
 'deceduti',
 'casi_da_sospetto_diagnostico',
 'casi_da_screening',
 'totale_casi',
 'tamponi',
 'casi_testati',
 'note',
 'ingressi_terapia_intensiva',
 'note_test',
 'note_casi',
 'totale_positivi_test_molecolare',
 'totale_positivi_test_antigenico_rapido',
 'tamponi_test_molecolare',
 'tamponi_test_antigenico_rapido',
 'codice_nuts_1',
 'codice_nuts_2']

In [5]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'casi_da_sospetto_diagnostico', 'casi_da_screening', 'note', 'note_test', 'note_casi']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,...,0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,...,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,...,0,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,...,18,148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9004,2021-04-27T17:00:00,19,Sicilia,1254,168,1422,24663,26085,-6,940,...,205585,3809167,1479597.0,4.0,205585.0,0.0,2253941.0,1555226.0,ITG,ITG1
9005,2021-04-27T17:00:00,9,Toscana,1449,257,1706,20060,21766,-905,522,...,224073,4038993,1980676.0,21.0,221246.0,2827.0,3280399.0,758594.0,ITI,ITI1
9006,2021-04-27T17:00:00,10,Umbria,164,33,197,2801,2998,13,96,...,54036,1142337,355527.0,4.0,54036.0,0.0,851029.0,291308.0,ITI,ITI2
9007,2021-04-27T17:00:00,2,Valle d'Aosta,48,8,56,727,783,-31,40,...,10786,111907,57146.0,0.0,10285.0,501.0,89201.0,22706.0,ITC,ITC2


In [6]:
# Compute most recent couple of dates
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = F.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2021-04-27', '2021-04-26')

In [7]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2021-04-27T17:00:00,1,Piemonte,2308,240,2548,14216,16764,-325,667,...,342086,3882563,1671265.0,12.0,325020.0,17066.0,2643662.0,1238901.0,ITC,ITC1
1,2021-04-27T17:00:00,2,Valle d'Aosta,48,8,56,727,783,-31,40,...,10786,111907,57146.0,0.0,10285.0,501.0,89201.0,22706.0,ITC,ITC2
2,2021-04-27T17:00:00,3,Lombardia,3819,582,4401,48650,53051,87,1369,...,797658,9313405,3782967.0,31.0,751954.0,45704.0,8044612.0,1268793.0,ITC,ITC4
3,2021-04-27T17:00:00,5,Veneto,1007,183,1190,21654,22844,-663,848,...,409213,7022634,1660537.0,25.0,398308.0,10905.0,4953650.0,2068984.0,ITH,ITH3
4,2021-04-27T17:00:00,6,Friuli Venezia Giulia,267,32,299,7745,8044,-221,186,...,104723,1821797,636890.0,2.0,90453.0,14270.0,1572804.0,248993.0,ITH,ITH4
5,2021-04-27T17:00:00,7,Liguria,504,66,570,5258,5828,-240,217,...,98571,1417916,579255.0,6.0,98571.0,0.0,1170491.0,247425.0,ITC,ITC3
6,2021-04-27T17:00:00,8,Emilia-Romagna,1833,246,2079,45724,47803,-720,723,...,366432,5493671,1764267.0,9.0,366143.0,289.0,4354948.0,1138723.0,ITH,ITH5
7,2021-04-27T17:00:00,9,Toscana,1449,257,1706,20060,21766,-905,522,...,224073,4038993,1980676.0,21.0,221246.0,2827.0,3280399.0,758594.0,ITI,ITI1
8,2021-04-27T17:00:00,10,Umbria,164,33,197,2801,2998,13,96,...,54036,1142337,355527.0,4.0,54036.0,0.0,851029.0,291308.0,ITI,ITI2
9,2021-04-27T17:00:00,11,Marche,457,66,523,5734,6257,-155,223,...,96766,1116163,668497.0,2.0,96766.0,0.0,1000190.0,115973.0,ITI,ITI3


In [8]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2021-04-26T17:00:00,1,Piemonte,2404,247,2651,14438,17089,-384,568,...,341419,3864827,1665337.0,7.0,324464.0,16955.0,2638590.0,1226237.0,ITC,ITC1
1,2021-04-26T17:00:00,2,Valle d'Aosta,46,8,54,760,814,22,53,...,10746,111275,56938.0,0.0,10269.0,477.0,88926.0,22349.0,ITC,ITC2
2,2021-04-26T17:00:00,3,Lombardia,3824,601,4425,48539,52964,-4658,872,...,796289,9277607,3769297.0,12.0,751195.0,45094.0,8027698.0,1249909.0,ITC,ITC4
3,2021-04-26T17:00:00,5,Veneto,1035,187,1222,22285,23507,195,535,...,408365,6987724,1656524.0,7.0,397668.0,10697.0,4939757.0,2047967.0,ITH,ITH3
4,2021-04-26T17:00:00,6,Friuli Venezia Giulia,286,37,323,7942,8265,-36,100,...,104537,1814450,634874.0,2.0,90332.0,14205.0,1568283.0,246167.0,ITH,ITH4
5,2021-04-26T17:00:00,7,Liguria,526,69,595,5473,6068,-57,200,...,98354,1410094,576951.0,2.0,98354.0,0.0,1165094.0,245000.0,ITC,ITC3
6,2021-04-26T17:00:00,8,Emilia-Romagna,1861,268,2129,46394,48523,-1958,936,...,365715,5462915,1761142.0,13.0,365427.0,288.0,4338082.0,1124833.0,ITH,ITH5
7,2021-04-26T17:00:00,9,Toscana,1466,262,1728,20943,22671,-175,737,...,223551,4017800,1974280.0,20.0,220738.0,2813.0,3270570.0,747230.0,ITI,ITI1
8,2021-04-26T17:00:00,10,Umbria,169,31,200,2785,2985,-40,25,...,53940,1134028,354552.0,2.0,53940.0,0.0,847833.0,286195.0,ITI,ITI2
9,2021-04-26T17:00:00,11,Marche,478,68,546,5866,6412,-168,123,...,96543,1112239,666408.0,1.0,96543.0,0.0,996473.0,115766.0,ITI,ITI3


In [9]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = df_raw_r_1
# df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
# df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

In [10]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = df_raw_r_0
# df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
# df_r_0.sort_values(by='codice_regione').reset_index()

In [11]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2,popolazione
0,2021-04-27T17:00:00,1,Piemonte,2308,240,2548,14216,16764,-325,667,...,3882563,1671265.0,12.0,325020.0,17066.0,2643662.0,1238901.0,ITC,ITC1,4.341375e+06
1,2021-04-27T17:00:00,2,Valle d'Aosta,48,8,56,727,783,-31,40,...,111907,57146.0,0.0,10285.0,501.0,89201.0,22706.0,ITC,ITC2,1.255010e+05
2,2021-04-27T17:00:00,3,Lombardia,3819,582,4401,48650,53051,87,1369,...,9313405,3782967.0,31.0,751954.0,45704.0,8044612.0,1268793.0,ITC,ITC4,1.010397e+07
3,2021-04-27T17:00:00,5,Veneto,1007,183,1190,21654,22844,-663,848,...,7022634,1660537.0,25.0,398308.0,10905.0,4953650.0,2068984.0,ITH,ITH3,4.907704e+06
4,2021-04-27T17:00:00,6,Friuli Venezia Giulia,267,32,299,7745,8044,-221,186,...,1821797,636890.0,2.0,90453.0,14270.0,1572804.0,248993.0,ITH,ITH4,1.211357e+06
5,2021-04-27T17:00:00,7,Liguria,504,66,570,5258,5828,-240,217,...,1417916,579255.0,6.0,98571.0,0.0,1170491.0,247425.0,ITC,ITC3,1.543127e+06
6,2021-04-27T17:00:00,8,Emilia-Romagna,1833,246,2079,45724,47803,-720,723,...,5493671,1764267.0,9.0,366143.0,289.0,4354948.0,1138723.0,ITH,ITH5,4.467118e+06
7,2021-04-27T17:00:00,9,Toscana,1449,257,1706,20060,21766,-905,522,...,4038993,1980676.0,21.0,221246.0,2827.0,3280399.0,758594.0,ITI,ITI1,3.722729e+06
8,2021-04-27T17:00:00,10,Umbria,164,33,197,2801,2998,13,96,...,1142337,355527.0,4.0,54036.0,0.0,851029.0,291308.0,ITI,ITI2,8.802850e+05
9,2021-04-27T17:00:00,11,Marche,457,66,523,5734,6257,-155,223,...,1116163,668497.0,2.0,96766.0,0.0,1000190.0,115973.0,ITI,ITI3,1.518400e+06


In [12]:
# Update icu csv data
df_icu_updated_r = pd.DataFrame.from_dict(F.icu_data())
df_icu_updated_r.to_csv(icu_regions_name, index=False)

In [13]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, left_on='denominazione_regione', right_on='regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2,popolazione,regione,ricoverati_area_non_critica,posti_letto_area_non_critica,ricoverati_terapia_intensiva,posti_letto_terapia_intensiva,posti_letto_terapia_intensiva_attivabili
0,2021-04-27T17:00:00,1,Piemonte,2308,240,2548,14216,16764,-325,667,...,1238901.0,ITC,ITC1,4.341375e+06,Piemonte,2308,5824,240,628,99
1,2021-04-27T17:00:00,2,Valle d'Aosta,48,8,56,727,783,-31,40,...,22706.0,ITC,ITC2,1.255010e+05,Valle d'Aosta,48,227,8,30,6
2,2021-04-27T17:00:00,3,Lombardia,3819,582,4401,48650,53051,87,1369,...,1268793.0,ITC,ITC4,1.010397e+07,Lombardia,3819,11497,582,1416,114
3,2021-04-27T17:00:00,5,Veneto,1007,183,1190,21654,22844,-663,848,...,2068984.0,ITH,ITH3,4.907704e+06,Veneto,1007,6000,183,1000,0
4,2021-04-27T17:00:00,6,Friuli Venezia Giulia,267,32,299,7745,8044,-221,186,...,248993.0,ITH,ITH4,1.211357e+06,Friuli Venezia Giulia,267,1277,32,175,0
5,2021-04-27T17:00:00,7,Liguria,504,66,570,5258,5828,-240,217,...,247425.0,ITC,ITC3,1.543127e+06,Liguria,504,1811,66,226,12
6,2021-04-27T17:00:00,8,Emilia-Romagna,1833,246,2079,45724,47803,-720,723,...,1138723.0,ITH,ITH5,4.467118e+06,Emilia-Romagna,1833,6538,246,760,0
7,2021-04-27T17:00:00,9,Toscana,1449,257,1706,20060,21766,-905,522,...,758594.0,ITI,ITI1,3.722729e+06,Toscana,1449,5033,257,625,17
8,2021-04-27T17:00:00,10,Umbria,164,33,197,2801,2998,13,96,...,291308.0,ITI,ITI2,8.802850e+05,Umbria,164,807,33,117,2
9,2021-04-27T17:00:00,11,Marche,457,66,523,5734,6257,-155,223,...,115973.0,ITI,ITI3,1.518400e+06,Marche,457,1251,66,210,54


In [14]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [15]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['posti_letto_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['posti_letto_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2021-04-27T17:00:00,15,Campania,1506,136,1642,90037,91679,-40,1654,...,21.94,-14,372,19689,14679.0,37,1657,8.40,0.02859,6.67
1,2021-04-27T17:00:00,3,Lombardia,3819,582,4401,48650,53051,87,1369,...,41.10,111,497,35798,13670.0,54,1228,3.82,0.01355,7.89
2,2021-04-27T17:00:00,16,Puglia,1726,227,1953,46708,48661,-67,1056,...,38.74,-65,579,14623,4550.0,37,1086,7.22,0.02635,5.76
3,2021-04-27T17:00:00,19,Sicilia,1254,168,1422,24663,26085,-6,940,...,20.12,0,-129,28762,9554.0,33,913,3.27,0.01892,4.14
4,2021-04-27T17:00:00,12,Lazio,2369,317,2686,43492,46178,-445,939,...,33.62,-389,-25,39745,18359.0,34,1350,2.36,0.01601,5.46
5,2021-04-27T17:00:00,5,Veneto,1007,183,1190,21654,22844,-663,848,...,18.30,-631,313,34910,4013.0,36,1475,2.43,0.01728,8.34
6,2021-04-27T17:00:00,8,Emilia-Romagna,1833,246,2079,45724,47803,-720,723,...,32.37,-670,-213,30756,3125.0,19,1418,2.35,0.01618,8.20
7,2021-04-27T17:00:00,1,Piemonte,2308,240,2548,14216,16764,-325,667,...,38.22,-222,99,17736,5928.0,27,965,3.76,0.01536,7.88
8,2021-04-27T17:00:00,9,Toscana,1449,257,1706,20060,21766,-905,522,...,41.12,-883,-215,21193,6396.0,34,1393,2.46,0.01402,6.02
9,2021-04-27T17:00:00,18,Calabria,474,45,519,14156,14675,-196,414,...,28.85,-187,229,3911,3357.0,9,601,10.59,0.02151,3.05


In [16]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [17]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'posti_letto_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,posti_letto_terapia_intensiva,saturazione_terapia_intensiva
0,Toscana,257,368,625,41.12
1,Lombardia,582,834,1416,41.10
2,Puglia,227,359,586,38.74
3,Piemonte,240,388,628,38.22
4,Lazio,317,626,943,33.62
5,Emilia-Romagna,246,514,760,32.37
6,Marche,66,144,210,31.43
7,Liguria,66,160,226,29.20
8,Calabria,45,111,156,28.85
9,Umbria,33,84,117,28.21


In [18]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['variazione_totale_positivi'] / (df['totale_positivi'] - df['variazione_totale_positivi'])).replace(np.inf, 0) * 100).round(2)
    return df.fillna(0)

In [19]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../../data/csv/regioni/covid19-veneto.csv
Aggiornato ../../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../../data/csv/regioni/covid19-liguria.csv
Aggiornato ../../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../../data/csv/regioni/covid19-toscana.csv
Aggiornato ../../data/csv/regioni/covid19-umbria.csv
Aggiornato ../../data/csv/regioni/covid19-marche.csv
Aggiornato ../../data/csv/regioni/covid19-lazio.csv
Aggiornato ../../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../../data/csv/regioni/covid19-molise.csv
Aggiornato ../../data/csv/regioni/covid19-campania.csv
Aggiornato ../../data/csv/regioni/covid19-puglia.csv
Aggiornato ../../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../../data/csv/regioni/covid19-calabria.csv
Aggiornato ../../data/csv/regioni/covid19-sicilia.csv


In [20]:
# df_tb = df_rd[df_rd.codice_regione > 20].reset_index(drop=True)
# df_tb

In [21]:
# Initialize df_taa
# df_taa = Functions.replace_bt_with_taa(df_tb[:2])
# for i in range(2, df_tb.shape[0], 2):
#     df_taa = df_taa.append(Functions.replace_bt_with_taa(df_tb[i:i+2]), ignore_index=True)
# df_taa = add_cols(df_taa)
# df_taa

In [22]:
# region_name = df_taa.denominazione_regione.iloc[0].lower()
# csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
# df_taa.to_csv(csv_out_region_name, index=False)
# print('Aggiornato {file}'.format(file=csv_out_region_name))